In [10]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [20]:
text = (open(r'Dataset/dataset.txt').read())
test = text.lower()
chars = sorted(list(set(text)))
char_to_int = dict((c,i) for i, c in enumerate(chars))

In [21]:
n_chars = len(text)
n_vocab = len(chars)
print("Total characters:",n_chars)
print("Total Vocab:",n_vocab)

Total characters: 22846
Total Vocab: 84
